In [1]:
import keras
import numpy as np
from sklearn import metrics
import h5py
from keras.models import model_from_json
from matplotlib import pyplot as plt 
from skimage import io
from keras import backend as K
from scipy.cluster.vq import vq, kmeans, whiten, kmeans2
from keras.models import load_model
from tempfile import TemporaryFile

Using TensorFlow backend.


In [5]:
# load the model from json and h5 (has both the model and the weight)
def loadModelJsonH5(model_name):
#     load the model architecture from the json file
    json_file = open(model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    loaded_model = model_from_json(loaded_model_json)
    
#     load the model parameters (weights) from the h5 file
    loaded_model.load_weights(model_name + ".h5")
    
#     print the summary of the model 
    loaded_model.summary()
    
    return loaded_model

In [14]:
# load the model from json (has the model only, no weight)
def loadModelJson(model_name):
#     load the model architecture from the json file
    json_file = open(model_name + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    loaded_model = model_from_json(loaded_model_json)
    
#     print the summary of the model 
#     loaded_model.summary()
    
    return loaded_model

In [ ]:
#load model from h5 (both model and weight)
def loadModelH5(model_name):
    filename = model_name + ".h5";
    print(filename)
    loaded_model = load_model(filename);
#     loaded_model.summary()
    return loaded_model

In [ ]:
#quantize the weights
#inputs are weights of a layer, number of cluster (how many numbers we can save, 8 usually works well) and 0/1 (1 if we want to vizualize)

def quantize_weights(wts, numClusters, viz_codeX):
    original_data=np.copy(wts)
    nz_idx=np.nonzero(original_data)
    nz_data=original_data[nz_idx]
    F=nz_data.flatten()
    F=F.reshape(-1,1)
    InitC=np.linspace(F.min(),F.max(),num=numClusters) #linear initialization is done. According to the paper this gives better result
    codebook, codeX=kmeans2(F, InitC.reshape(-1,1), minit='matrix')
    
    if viz_codeX==1:
        print(codebook)
        print(len(codeX))
        # edges_hist=[x for x in range(numClusters+1)]
        # frq, edges = np.histogram(codeX,edges_hist)
        # print(frq,edges)
        # fig, ax = plt.subplots()
        # ax.bar(edges[:-1], frq, width=np.diff(edges), ec="k", align="edge")
        # plt.title("cluster value histogram")
        # plt.show()
    return codebook, codeX

In [19]:
# get the weight from the codeX and codeVal
def create_codeVal(codeX, codebook, wts_shape):
    code_val=np.zeros(codeX.shape)
    for idx,val in enumerate(codeX):
        code_val[idx]=codebook[val]
    new_wts=code_val.reshape(wts_shape)
    return new_wts

In [ ]:
#Sample code for loading a model and quantize_weights
# def sample_quantize_weights

# model = loadModelJsonH5("Models/model3")
def sample_quantize(modelname):
    codebooks = []
    codeXs = []
    cluster_number = 8
    model = loadModelH5(modelname)
    for layer in model.layers:
        print(layer)
        existing_weight = layer.get_weights()
        existing_weight_np = np.asarray(existing_weight)
        print(existing_weight_np.shape)
    #     If convolution layer then there is both weight and bias index 0 is weight and index 1 is bias
        if existing_weight_np.shape == (2,):
            for i in range (0,2):
                codebook, codeX = quantize_weights(existing_weight_np[i], cluster_number, 0)
                codebooks.append(codebook)
                codeXs.append(codeX)
        elif existing_weight_np.shape != (0,) :
            codebook, codeX = quantize_weights(existing_weight_np, cluster_number, 0)
            codebooks.append(codebook)
            codeXs.append(codeX)
    return codebooks, codeXs
#         by saving the codebook, codeX and the shape of the weight we can save space. 


# this calls the sample function
codebook, codeX = sample_quantize("tr")
#Now we will save them 
np.save("codebook_file.npy", codebook)

codeX_file = TemporaryFile()
np.save("codeX_file.npy", codeX)

In [30]:
# Read the quantized value and get the new weights then build the model

def sample_create_codeval(model_name, codeX_name, codebooks_name):
    model = loadModelJson(model_name)
    codeXs = np.load(codeX_name)
    codebooks = np.load(codebooks_name)
    index = 0
    for layer in model.layers:
            print(layer)
            existing_weight = layer.get_weights()
            existing_weight_np = np.asarray(existing_weight)
#             print(existing_weight_np.shape)

    #         if convolution layer we update both weight and bias
            if existing_weight_np.shape == (2,):
                new_w = []
                for i in range (0,2):
                    codebook = codebooks[index]
                    codeX = codeXs[index]
                    new_weight = create_codeVal(codeX, codebook, existing_weight_np[i].shape)
                    index += 1
                    new_w.append(new_weight)
                new_weights = np.asarray(new_w)
    #             all other layers which have parameters
            elif existing_weight_np.shape != (0,) :
                codebook = codebooks[index]
                codeX = codeXs[index]
                new_weights = create_codeVal(codeX, codebook, existing_weight_np.shape)
                index += 1
    #             for any layer with parameter we update the parametes
            if existing_weight_np.shape != (0,) :
                layer.set_weights(new_weights)

#             existing_weight = layer.get_weights()
#             existing_weight_np = np.asarray(existing_weight)
#             print(existing_weight_np.shape)
    return model
            
# we are calling the sample here which will give us the final model 
model = sample_create_codeval("model4", "codeX_file.npy", "codebook_file.npy")